In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree, svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
from scipy.stats import norm
# import seaborn as sns
# import math
# from mpl_toolkits.mplot3d import Axes3D

train_df = pd.read_csv("./Assignment3-TrainingData-ModifiedV2.csv")
predict_df = pd.read_csv("./Assignment3-UnknownData.csv")

In [15]:
# norm_train = {}
# attributes = train_df.columns

# for attribute in attributes:
#     if (attribute != "row ID" and attribute != "QUALIFIED"):
#         minMax_scaler = MinMaxScaler()
#         minMax_scaler.fit_transform(train_df[[attribute]])
        
#         norm_train[attribute] = minMax_scaler.fit_transform(train_df[[attribute]])
        
# for attribute in attributes:
#     if (attribute != "row ID" and attribute != "QUALIFIED"):
#         for y in (0, len(norm_train[attribute]-1)):
#             norm_train[attribute][y] = norm_train[attribute][y][0]
            
# print(norm_train)

In [16]:
def savePrediction(rowID, prediction, fileName):
    pd.DataFrame({
        "Row ID": rowID,
        "Predict-Qualified": prediction,
    }).to_csv(fileName, index=False)

In [17]:
inputs = train_df.drop("QUALIFIED", axis="columns").drop("row ID", axis="columns")
target = train_df["QUALIFIED"]
x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=1)

In [18]:
def evaluatePrediction(prediction, reality):
    confusionMatrix = confusion_matrix(prediction, reality)

    total = confusionMatrix[0][0] + confusionMatrix[1][0] + confusionMatrix[0][1] + confusionMatrix[1][1]
    accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / total
    
    return {
        "confusionMatrix": confusionMatrix,
        "accuracy": accuracy
    }

In [19]:
def calcPr(confusionMatrix):
    tpr = confusionMatrix[0][0] / (confusionMatrix[0][0] + confusionMatrix[1][0])
    fpr = confusionMatrix[0][1] / (confusionMatrix[0][1] + confusionMatrix[1][1])
    return [fpr, tpr]

In [20]:
def makePrediction(classifier, seed):
    x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=seed)
    prediction = classifier()
    prediction = prediction.fit(x_train, y_train)
    qualifiedPrediction = prediction.predict(x_test)
    
    evaluation = evaluatePrediction(qualifiedPrediction, y_test)
    pr = calcPr(evaluation["confusionMatrix"])
    
    return {
        "qualifiedPrediction": qualifiedPrediction,
        "fpr": pr[0],
        "tpr": pr[1],
        "evaluate": evaluation
    }

In [21]:
print(makePrediction(tree.DecisionTreeClassifier, 1))

{'qualifiedPrediction': array([0, 0, 1, ..., 0, 1, 0], dtype=int64), 'fpr': 0.15371302706104467, 'tpr': 0.8859060402684564, 'evaluate': {'confusionMatrix': array([[7656,  977],
       [ 986, 5379]], dtype=int64), 'accuracy': 0.8691158821176157}}


In [22]:
# classifiers = [tree.DecisionTreeClassifier, svm.SVC, RandomForestClassifier, KNeighborsClassifier(n_neighbors=9), MLPClassifier(solver='lbfgs', activation="logistic", hidden_layer_sizes=(21))]
# cName = ["Decision Tree", "SVM", "Random Forest", "kNN", "Neural Network"]
# i = 0
# accuracy = {}

# for classifier in classifiers:
#     print(f"Classifier: {cName[i]}")
#     accuracy[cName[i]] = []
    
#     for i in range(1, 5 if i == 1 else ):
#         treeResult = makePrediction(classifier, i)
#         sum += treeResult["evaluate"]["accuracy"]
#         accuracy.append(treeResult["evaluate"]["accuracy"])
        
#     i += 1

In [23]:
# for i in range(1, 30):
#     x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=i)

In [24]:
# auc_tree = auc(tree_fpr, tree_tpr)

In [25]:
svmPredict = svm.SVC()
svmPredict.fit(inputs, target)
svm_qPrediction = svmPredict.predict(inputs)

In [26]:
# For whole dataset
savePrediction(train_df["row ID"], svm_qPrediction, "SVMPrediction.csv")

In [27]:
evaluatePrediction(svm_qPrediction, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [74987, 14998]

In [ ]:
svmScore = svmPredict.decision_function(x_test)
svm_fpr, svm_tpr, svm_threshold = roc_curve(y_test, svmScore)

In [ ]:
print(auc(svm_fpr, svm_tpr))

In [ ]:
# mlpPredict = MLPClassifier(solver="lbfgs", alpha)

In [ ]:
randForestPredict = RandomForestClassifier(n_estimators=10)
randForestPredict = randForestPredict.fit(x_train, y_train)
randForestPrediction = randForestPredict.predict(x_test)

evaluatePrediction(randForestPrediction, y_test)

In [ ]:
randForestPredict_fpr, randForestPredict_tpr, randForestPredict_threshold = roc_curve(y_test, randForestPrediction)

In [ ]:
# https://towardsdatascience.com/knn-using-scikit-learn-c6bed765be75
scores = {}
scores_list = []

for k in range(1, 26):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    scores[k] = {
        "score": accuracy_score(y_test, y_pred),
        "prediction": y_pred
    }
    scores_list.append(accuracy_score(y_test, y_pred))
    
print(scores[scores_list.index(max(scores_list))+1])

In [ ]:
print(scores_list)

In [ ]:
kNN_prediction = scores[scores_list.index(max(scores_list))+1]["prediction"]
evaluatePrediction(kNN_prediction, y_test)
kNNPredict_fpr, kNNPredict_tpr, kNNPredict_threshold = roc_curve(y_test, kNN_prediction)

In [ ]:
neuralPredict = MLPClassifier(solver='lbfgs', activation="logistic", hidden_layer_sizes=(21), random_state=1)
neuralPredict = neuralPredict.fit(x_train, y_train)
neuralPrediction = neuralPredict.predict(x_test)

evaluatePrediction(neuralPrediction, y_test)

In [ ]:
def displayROC(fprsAndTprs):
    for rate in fprsAndTprs:
        auc = auc(rate[0], rate[1])
        plt.plot(fpr, tpr, label="SVM (AUC = %0.3f)" % svm)

    plt.plot([0,1], [0,1], label="Useless Test")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1, 0.75))
    plt.show()

In [ ]:
auc_svm = auc(svm_fpr, svm_tpr)
auc_randForest = auc(randForestPredict_fpr, randForestPredict_tpr)
auc_kNN = auc(kNNPredict_fpr, kNNPredict_tpr)

plt.plot(svm_fpr, svm_tpr, label="SVM (AUC = %0.3f)" % auc_svm)
plt.plot(tree_fpr, tree_tpr, label="CART (AUC = %0.3f)" % auc_tree)
plt.plot(randForestPredict_fpr, randForestPredict_tpr, label="Random Forest (AUC = %0.3f)" % auc_randForest)
plt.plot(kNNPredict_fpr, kNNPredict_tpr, label="K Nearest Neighbours (AUC = %0.3f)" % auc_kNN)
plt.plot([0,1], [0,1], label="Useless Test")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.grid(True)
plt.legend(bbox_to_anchor=(1, 0.75))
plt.show()